# Using Cohere Connectors For RAG

This notebook covers how to get started with Cohere connectors. More information on documentation can be found here https://docs.cohere.com/docs/connectors.

## Installation

In [ ]:
# install package
!pip install -U langchain-cohere langchain-core

## Environment Setup

Make sure to set the following environment variables:

- TODO: fill out relevant environment variables or secrets

## Usage

In [2]:
from langchain_core.messages.human import HumanMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    MessagesPlaceholder,
)
from langchain_core.runnables import RunnablePassthrough
from langchain_cohere.chat_models import ChatCohere

In [3]:
import getpass
import os

os.environ["COHERE_API_KEY"] = getpass.getpass()
llm = ChatCohere(model_name="command-r")

In [4]:
def get_connectors(_):
    return ["web-search"]

In [5]:
def format_input_msgs(input):
    return [
        HumanMessage(
            input["message"],
            additional_kwargs={
                "connectors": input.get("connectors", None),
            },
        )
    ]

prompt = ChatPromptTemplate.from_messages(
    [
        # ("system", "You are a helpful assistant that ..."),
        MessagesPlaceholder("input_msgs"),
    ]
)

In [6]:
chain = (
    {"message": RunnablePassthrough(), "connectors": get_connectors}
    | RunnablePassthrough()
    | {"input_msgs": format_input_msgs}
    | prompt
    | llm
)

In [14]:
response = chain.invoke("What are the top headlines in the world today?")
print(response.content)

Here are some of the top headlines around the world today:

## Israel-Palestine Conflict
- Gaza is facing imminent famine as more than a million people face 'catastrophic' levels of hunger, aid agencies warn. 
- Israeli military launches a new raid at Gaza's largest hospital, once the epicenter of the fighting.
- US food aid to Gaza is criticized for being too little and ineffective.

## Russia-Ukraine War
- Russia pounds Ukrainian power facilities as the Kremlin embraces 'war' rhetoric.
- Ukraine says power line to the Zaporizhzhia nuclear plant has been fixed after blackout risk.

## Other International Headlines
- Hong Kong passes a second national security law, widening crackdown powers and aligning the city with mainland China. 
- China accuses Evergrande Group, the country's second-biggest property developer, of fraud worth $78 billion, in what could be the biggest fraud in Chinese history.
- Finland has been named the happiest country in the world for the seventh year in a row, 

## Printing results with citations

In [15]:
"""
HELPER CODE FOR CITATIONS FORMATTING
"""
def insert_citations_in_order(text, citations, documents):
    """
    A helper function to pretty print citations.
    """

    citations_reference = {}
    for index, doc in enumerate(documents):
        citations_reference[index] = doc

    offset = 0
    # Process citations in the order they were provided
    for citation in citations:
        # Adjust start/end with offset
        start, end = citation.start + offset, citation.end + offset
        citation_numbers = []
        for doc_id in citation.document_ids:
            for citation_index, doc in citations_reference.items():
                if doc["id"] == doc_id:
                    citation_numbers.append(citation_index)
        references = "(" + ", ".join("[{}]".format(num) for num in citation_numbers) + ")"
        modification = f'{text[start:end]} {references}'
        # Replace the cited text with its bolded version + placeholder
        text = text[:start] + modification + text[end:]
        # Update the offset for subsequent replacements
        offset += len(modification) - (end - start)

    # Add the citations at the bottom of the text
    text_with_citations = f'{text}'
    citations_reference = ["[{}]: {}".format(x["id"], x["snippet"]) for x in citations_reference.values()]

    return text_with_citations, "\n".join(citations_reference)

In [16]:
cited_response, citations_reference = insert_citations_in_order(response.content, response.response_metadata["citations"], response.response_metadata["documents"])
print(cited_response)
print("\n")
print("References:")
print(citations_reference)

Here are some of the top headlines around the world today:

## Israel-Palestine Conflict
- Gaza is facing imminent famine ([0]) as more than a million people face 'catastrophic' levels of hunger ([0]), aid agencies warn. ([0]) 
- Israeli military launches a new raid at Gaza's largest hospital ([0]), once the epicenter of the fighting. ([0])
- US food aid to Gaza is criticized for being too little and ineffective. ([1])

## Russia-Ukraine War
- Russia pounds Ukrainian power facilities as the Kremlin embraces 'war' rhetoric. ([2], [3])
- Ukraine says power line to the Zaporizhzhia nuclear plant has been fixed ([2]) after blackout risk. ([2])

## Other International Headlines
- Hong Kong passes a second national security law ([0]), widening crackdown powers and aligning the city with mainland China. ([0]) 
- China accuses Evergrande Group ([3], [0]), the country's second-biggest property developer ([0]), of fraud worth $78 billion ([0]), in what could be the biggest fraud in Chinese histo

In [ ]:
cited_response, citations_reference = insert_citations_in_order(response.content, response.response_metadata["citations"], response.response_metadata["documents"])
print(cited_response)
print("\n")
print("References:")
print(citations_reference)

Here are some of the top headlines around the world today:

## Israel-Palestine Conflict
- Gaza is facing imminent famine ([0]) as more than a million people face 'catastrophic' levels of hunger ([0]), aid agencies warn. ([0]) 
- Israeli military launches a new raid at Gaza's largest hospital ([0]), once the epicenter of the fighting. ([0])
- US food aid to Gaza is criticized for being too little and ineffective. ([1])

## Russia-Ukraine War
- Russia pounds Ukrainian power facilities as the Kremlin embraces 'war' rhetoric. ([2], [3])
- Ukraine says power line to the Zaporizhzhia nuclear plant has been fixed ([2]) after blackout risk. ([2])

## Other International Headlines
- Hong Kong passes a second national security law ([0]), widening crackdown powers and aligning the city with mainland China. ([0]) 
- China accuses Evergrande Group ([3], [0]), the country's second-biggest property developer ([0]), of fraud worth $78 billion ([0]), in what could be the biggest fraud in Chinese histo